In [14]:
import pyodbc

In [15]:
pyodbc.drivers()

['SQL Server',
 'ODBC Driver 17 for SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)']

In [16]:
from sqlalchemy import create_engine
import urllib

server = 'DESKTOP-6MJU11J'
database = 'iotdata_events' 

params = urllib.parse.quote_plus(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"Trusted_Connection=yes;"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# Example test

with engine.connect() as conn:
    result = conn.execute("SELECT GETDATE()")
    for row in result:
        print(row)


(datetime.datetime(2025, 4, 19, 10, 21, 36, 160000),)


In [17]:
# Load data
import pandas as pd
table = "FireEvents" 
df = pd.read_sql(f"SELECT * FROM {table}", engine)
print(df.shape)
df.head()

(940083, 12)


,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,IDENTIFIER,RECORDEDAT,SITEID,STATE,SYSTEMGENERATED,WILLSHOWINAPP,MESSAGE
0,2024-08-07 17:56:29.842,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,None,O0uFfYqPcl4PMU3uXxpe,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2024-08-07 17:56:28.957,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,True,"{\n ""descriptor"": ""initialising rl03 back of ..."
1,2024-08-14 10:49:42.359,n7Tktl6xJA2qDV8RcMpf,yPHI0SRu7cRZ2YNmfDPG,None,xByYZSjb4uJqpLZkLwCa,"{\n ""device"": ""191"",\n ""group"": null,\n ""lo...",2024-08-14 10:49:40.278,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,True,"{\n ""descriptor"": ""normal rl03 smoke zone 2 r..."
2,2024-09-03 17:05:31.186,n7Tktl6xJA2qDV8RcMpf,hsBtiG1y58ChVue6Mstj,None,EEL0bHavix9GmwdKvd0p,"{\n ""device"": ""92"",\n ""group"": null,\n ""loo...",2024-09-03 17:05:16.008,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,True,"{\n ""descriptor"": ""device missing rl06 smoke ..."
3,2025-01-09 03:27:26.923,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,None,4RVo9ZSGMWFzHhvtUDcd,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2025-01-09 03:27:24.867,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,True,"{\n ""descriptor"": ""normal rl03 back of house ..."
4,2024-08-28 12:18:26.508,n7Tktl6xJA2qDV8RcMpf,5r2EosP2Vnb9cMJsQUA9,None,91S1PYh6HlGxdBqqMf9S,"{\n ""device"": ""136"",\n ""group"": null,\n ""lo...",2024-08-28 12:18:25.776,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,True,"{\n ""descriptor"": ""device missing node 1 ffcp..."


In [18]:
import json

In [19]:
# Step 2: Load data from the SQL Server into a DataFrame
#table = "FireEvents"  # Name of your SQL table
##df = pd.read_sql(f"SELECT * FROM {table}", engine)

# Display the shape (rows, columns) of the loaded DataFrame
print("Original DataFrame shape:", df.shape)

# Show the first few rows to preview the structure
##df.head()

Original DataFrame shape: (940083, 12)


In [20]:
# Step 3: Check structure of one JSON entry to verify format
print("Sample JSON from EventDetails column:")
print(df['IDENTIFIER'].iloc[0])

# Step 3: Check structure of one JSON entry to verify format
print("Sample JSON from SITE column:")
print(df['STATE'].iloc[0])

# Step 3: Check structure of one JSON entry to verify format
print("Sample JSON from MESSAGE column:")
print(df['MESSAGE'].iloc[0])

# Step 3: Check structure of one JSON entry to verify format
print("Sample JSON from DURATION column:")
print(df['DURATION'].iloc[0])

Sample JSON from EventDetails column:
{
  "device": "172",
  "group": null,
  "loop": "26",
  "module": null,
  "node": "9",
  "sensor": "0",
  "zone": "109"
}
Sample JSON from SITE column:
{
  "alarm": "off",
  "fault": "off",
  "faultCode": null,
  "isolate": "off",
  "preAlarm": "off",
  "sensor": null
}
Sample JSON from MESSAGE column:
{
  "descriptor": "initialising rl03 back of house rl03 z1 corridor mcp 3012a",
  "detector": "Z109N9L26D172S0",
  "notification": "z109n9l26d172s0 - m214 - SC1 retail",
  "raw": "initialising rl03 back of house rl03 z1 corridor mcp 3012a",
  "state": "NNNN"
}
Sample JSON from DURATION column:
None


In [21]:
df=df.join(df['IDENTIFIER'].apply(json.loads).apply(pd.Series))

In [22]:
print(df.shape)
df.head()

(940083, 19)


,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,IDENTIFIER,RECORDEDAT,SITEID,STATE,SYSTEMGENERATED,WILLSHOWINAPP,MESSAGE,device,group,loop,module,node,sensor,zone
0,2024-08-07 17:56:29.842,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,None,O0uFfYqPcl4PMU3uXxpe,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2024-08-07 17:56:28.957,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,True,"{\n ""descriptor"": ""initialising rl03 back of ...",172,None,26,None,9,0,109
1,2024-08-14 10:49:42.359,n7Tktl6xJA2qDV8RcMpf,yPHI0SRu7cRZ2YNmfDPG,None,xByYZSjb4uJqpLZkLwCa,"{\n ""device"": ""191"",\n ""group"": null,\n ""lo...",2024-08-14 10:49:40.278,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,True,"{\n ""descriptor"": ""normal rl03 smoke zone 2 r...",191,None,26,None,9,0,102
2,2024-09-03 17:05:31.186,n7Tktl6xJA2qDV8RcMpf,hsBtiG1y58ChVue6Mstj,None,EEL0bHavix9GmwdKvd0p,"{\n ""device"": ""92"",\n ""group"": null,\n ""loo...",2024-09-03 17:05:16.008,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,True,"{\n ""descriptor"": ""device missing rl06 smoke ...",92,None,13,None,5,0,191
3,2025-01-09 03:27:26.923,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,None,4RVo9ZSGMWFzHhvtUDcd,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2025-01-09 03:27:24.867,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,True,"{\n ""descriptor"": ""normal rl03 back of house ...",172,None,26,None,9,0,109
4,2024-08-28 12:18:26.508,n7Tktl6xJA2qDV8RcMpf,5r2EosP2Vnb9cMJsQUA9,None,91S1PYh6HlGxdBqqMf9S,"{\n ""device"": ""136"",\n ""group"": null,\n ""lo...",2024-08-28 12:18:25.776,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,True,"{\n ""descriptor"": ""device missing node 1 ffcp...",136,None,3,None,1,0,801


In [23]:
df = df.join(df['STATE'].apply(json.loads).apply(pd.Series), rsuffix='_from_state')

In [24]:
print(df.shape)
df.head()

(940083, 25)


,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,IDENTIFIER,RECORDEDAT,SITEID,STATE,SYSTEMGENERATED,...,module,node,sensor,zone,alarm,fault,faultCode,isolate,preAlarm,sensor_from_state
0,2024-08-07 17:56:29.842,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,None,O0uFfYqPcl4PMU3uXxpe,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2024-08-07 17:56:28.957,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,9,0,109,off,off,None,off,off,None
1,2024-08-14 10:49:42.359,n7Tktl6xJA2qDV8RcMpf,yPHI0SRu7cRZ2YNmfDPG,None,xByYZSjb4uJqpLZkLwCa,"{\n ""device"": ""191"",\n ""group"": null,\n ""lo...",2024-08-14 10:49:40.278,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,9,0,102,off,off,None,off,off,None
2,2024-09-03 17:05:31.186,n7Tktl6xJA2qDV8RcMpf,hsBtiG1y58ChVue6Mstj,None,EEL0bHavix9GmwdKvd0p,"{\n ""device"": ""92"",\n ""group"": null,\n ""loo...",2024-09-03 17:05:16.008,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,...,None,5,0,191,unknown,on,"{'panel': 'AXIS', 'text': None, 'value': None}",unknown,unknown,None
3,2025-01-09 03:27:26.923,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,None,4RVo9ZSGMWFzHhvtUDcd,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2025-01-09 03:27:24.867,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,9,0,109,off,off,None,off,off,None
4,2024-08-28 12:18:26.508,n7Tktl6xJA2qDV8RcMpf,5r2EosP2Vnb9cMJsQUA9,None,91S1PYh6HlGxdBqqMf9S,"{\n ""device"": ""136"",\n ""group"": null,\n ""lo...",2024-08-28 12:18:25.776,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,...,None,1,0,801,unknown,on,"{'panel': 'AXIS', 'text': None, 'value': None}",unknown,unknown,None


In [25]:
df=df.join(df['MESSAGE'].apply(json.loads).apply(pd.Series))


In [26]:
print(df.shape)
df.head()

(940083, 30)


,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,IDENTIFIER,RECORDEDAT,SITEID,STATE,SYSTEMGENERATED,...,fault,faultCode,isolate,preAlarm,sensor_from_state,descriptor,detector,notification,raw,state
0,2024-08-07 17:56:29.842,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,None,O0uFfYqPcl4PMU3uXxpe,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2024-08-07 17:56:28.957,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,off,None,off,off,None,initialising rl03 back of house rl03 z1 corrid...,Z109N9L26D172S0,z109n9l26d172s0 - m214 - SC1 retail,initialising rl03 back of house rl03 z1 corrid...,NNNN
1,2024-08-14 10:49:42.359,n7Tktl6xJA2qDV8RcMpf,yPHI0SRu7cRZ2YNmfDPG,None,xByYZSjb4uJqpLZkLwCa,"{\n ""device"": ""191"",\n ""group"": null,\n ""lo...",2024-08-14 10:49:40.278,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,off,None,off,off,None,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,Z102N9L26D191S0,z102n9l26d191s0 - m214 - SC1 retail,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,NNNN
2,2024-09-03 17:05:31.186,n7Tktl6xJA2qDV8RcMpf,hsBtiG1y58ChVue6Mstj,None,EEL0bHavix9GmwdKvd0p,"{\n ""device"": ""92"",\n ""group"": null,\n ""loo...",2024-09-03 17:05:16.008,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,...,on,"{'panel': 'AXIS', 'text': None, 'value': None}",unknown,unknown,None,device missing rl06 smoke zone 1 rl06 sz1 77c ...,Z191N5L13D92S0,z191n5l13d92s0 - m214 - SC1 retail,device missing rl06 smoke zone 1 rl06 sz1 77c ...,??F?
3,2025-01-09 03:27:26.923,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,None,4RVo9ZSGMWFzHhvtUDcd,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2025-01-09 03:27:24.867,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,off,None,off,off,None,normal rl03 back of house rl03 z1 corridor mcp...,Z109N9L26D172S0,z109n9l26d172s0 - SC1 retail - m214,normal rl03 back of house rl03 z1 corridor mcp...,NNNN
4,2024-08-28 12:18:26.508,n7Tktl6xJA2qDV8RcMpf,5r2EosP2Vnb9cMJsQUA9,None,91S1PYh6HlGxdBqqMf9S,"{\n ""device"": ""136"",\n ""group"": null,\n ""lo...",2024-08-28 12:18:25.776,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,...,on,"{'panel': 'AXIS', 'text': None, 'value': None}",unknown,unknown,None,device missing node 1 ffcp modules eaf-b01-1 s...,Z801N1L3D136S0,z801n1l3d136s0 - m214 - SC1 retail,device missing node 1 ffcp modules eaf-b01-1 s...,??F?


In [27]:
print(df.shape)

(940083, 30)


In [28]:
df['DURATION'] = df['DURATION'].fillna('{}')


In [29]:
print(df['DURATION'].dtype)
print(df['DURATION'].head())

object
0    {}
1    {}
2    {}
3    {}
4    {}
Name: DURATION, dtype: object


In [30]:
df_duration_expanded = df['DURATION'].apply(json.loads).apply(pd.Series, dtype='object')


In [31]:

#Rename columns with a prefix
df_duration_expanded = df_duration_expanded.rename(columns=lambda x: f'duration_{x}')

In [35]:
df = df.join(df_duration_expanded)

In [36]:
print (df.shape)
df.head()

(940083, 34)


,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,IDENTIFIER,RECORDEDAT,SITEID,STATE,SYSTEMGENERATED,...,sensor_from_state,descriptor,detector,notification,raw,state,duration_alarm,duration_fault,duration_isolate,duration_preAlarm
0,2024-08-07 17:56:29.842,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},O0uFfYqPcl4PMU3uXxpe,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2024-08-07 17:56:28.957,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,initialising rl03 back of house rl03 z1 corrid...,Z109N9L26D172S0,z109n9l26d172s0 - m214 - SC1 retail,initialising rl03 back of house rl03 z1 corrid...,NNNN,NaN,NaN,NaN,NaN
1,2024-08-14 10:49:42.359,n7Tktl6xJA2qDV8RcMpf,yPHI0SRu7cRZ2YNmfDPG,{},xByYZSjb4uJqpLZkLwCa,"{\n ""device"": ""191"",\n ""group"": null,\n ""lo...",2024-08-14 10:49:40.278,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,Z102N9L26D191S0,z102n9l26d191s0 - m214 - SC1 retail,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,NNNN,NaN,NaN,NaN,NaN
2,2024-09-03 17:05:31.186,n7Tktl6xJA2qDV8RcMpf,hsBtiG1y58ChVue6Mstj,{},EEL0bHavix9GmwdKvd0p,"{\n ""device"": ""92"",\n ""group"": null,\n ""loo...",2024-09-03 17:05:16.008,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,...,None,device missing rl06 smoke zone 1 rl06 sz1 77c ...,Z191N5L13D92S0,z191n5l13d92s0 - m214 - SC1 retail,device missing rl06 smoke zone 1 rl06 sz1 77c ...,??F?,NaN,NaN,NaN,NaN
3,2025-01-09 03:27:26.923,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},4RVo9ZSGMWFzHhvtUDcd,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2025-01-09 03:27:24.867,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,normal rl03 back of house rl03 z1 corridor mcp...,Z109N9L26D172S0,z109n9l26d172s0 - SC1 retail - m214,normal rl03 back of house rl03 z1 corridor mcp...,NNNN,NaN,NaN,NaN,NaN
4,2024-08-28 12:18:26.508,n7Tktl6xJA2qDV8RcMpf,5r2EosP2Vnb9cMJsQUA9,{},91S1PYh6HlGxdBqqMf9S,"{\n ""device"": ""136"",\n ""group"": null,\n ""lo...",2024-08-28 12:18:25.776,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,...,None,device missing node 1 ffcp modules eaf-b01-1 s...,Z801N1L3D136S0,z801n1l3d136s0 - m214 - SC1 retail,device missing node 1 ffcp modules eaf-b01-1 s...,??F?,NaN,NaN,NaN,NaN


In [37]:
df.loc[2, 'state']

'??F?'

In [38]:
df.loc[2, 'state']

'??F?'

In [40]:
df['duration_isolate']

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
            ...   
940078         NaN
940079         NaN
940080         NaN
940081    PT0.328S
940082        None
Name: duration_isolate, Length: 940083, dtype: object

In [41]:
!pip install isodate

In [42]:
from isodate import parse_duration
def safe_parse_duration(x):
    try:
        if pd.isnull(x):
            return pd.NaT
        return parse_duration(x)
    except:
        return pd.NaT
df['duration_isolate'] = df['duration_isolate'].apply(safe_parse_duration)



In [43]:
df['duration_isolate']


0                           NaT
1                           NaT
2                           NaT
3                           NaT
4                           NaT
                  ...          
940078                      NaT
940079                      NaT
940080                      NaT
940081   0 days 00:00:00.328000
940082                      NaT
Name: duration_isolate, Length: 940083, dtype: timedelta64[ns]

In [44]:
df.head()

,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,IDENTIFIER,RECORDEDAT,SITEID,STATE,SYSTEMGENERATED,...,sensor_from_state,descriptor,detector,notification,raw,state,duration_alarm,duration_fault,duration_isolate,duration_preAlarm
0,2024-08-07 17:56:29.842,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},O0uFfYqPcl4PMU3uXxpe,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2024-08-07 17:56:28.957,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,initialising rl03 back of house rl03 z1 corrid...,Z109N9L26D172S0,z109n9l26d172s0 - m214 - SC1 retail,initialising rl03 back of house rl03 z1 corrid...,NNNN,NaN,NaN,NaT,NaN
1,2024-08-14 10:49:42.359,n7Tktl6xJA2qDV8RcMpf,yPHI0SRu7cRZ2YNmfDPG,{},xByYZSjb4uJqpLZkLwCa,"{\n ""device"": ""191"",\n ""group"": null,\n ""lo...",2024-08-14 10:49:40.278,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,Z102N9L26D191S0,z102n9l26d191s0 - m214 - SC1 retail,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,NNNN,NaN,NaN,NaT,NaN
2,2024-09-03 17:05:31.186,n7Tktl6xJA2qDV8RcMpf,hsBtiG1y58ChVue6Mstj,{},EEL0bHavix9GmwdKvd0p,"{\n ""device"": ""92"",\n ""group"": null,\n ""loo...",2024-09-03 17:05:16.008,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,...,None,device missing rl06 smoke zone 1 rl06 sz1 77c ...,Z191N5L13D92S0,z191n5l13d92s0 - m214 - SC1 retail,device missing rl06 smoke zone 1 rl06 sz1 77c ...,??F?,NaN,NaN,NaT,NaN
3,2025-01-09 03:27:26.923,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},4RVo9ZSGMWFzHhvtUDcd,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2025-01-09 03:27:24.867,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,normal rl03 back of house rl03 z1 corridor mcp...,Z109N9L26D172S0,z109n9l26d172s0 - SC1 retail - m214,normal rl03 back of house rl03 z1 corridor mcp...,NNNN,NaN,NaN,NaT,NaN
4,2024-08-28 12:18:26.508,n7Tktl6xJA2qDV8RcMpf,5r2EosP2Vnb9cMJsQUA9,{},91S1PYh6HlGxdBqqMf9S,"{\n ""device"": ""136"",\n ""group"": null,\n ""lo...",2024-08-28 12:18:25.776,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,...,None,device missing node 1 ffcp modules eaf-b01-1 s...,Z801N1L3D136S0,z801n1l3d136s0 - m214 - SC1 retail,device missing node 1 ffcp modules eaf-b01-1 s...,??F?,NaN,NaN,NaT,NaN


In [45]:
df['duration_isolate'] = df['duration_isolate'].apply(safe_parse_duration)


In [46]:
df.head()

,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,IDENTIFIER,RECORDEDAT,SITEID,STATE,SYSTEMGENERATED,...,sensor_from_state,descriptor,detector,notification,raw,state,duration_alarm,duration_fault,duration_isolate,duration_preAlarm
0,2024-08-07 17:56:29.842,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},O0uFfYqPcl4PMU3uXxpe,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2024-08-07 17:56:28.957,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,initialising rl03 back of house rl03 z1 corrid...,Z109N9L26D172S0,z109n9l26d172s0 - m214 - SC1 retail,initialising rl03 back of house rl03 z1 corrid...,NNNN,NaN,NaN,NaT,NaN
1,2024-08-14 10:49:42.359,n7Tktl6xJA2qDV8RcMpf,yPHI0SRu7cRZ2YNmfDPG,{},xByYZSjb4uJqpLZkLwCa,"{\n ""device"": ""191"",\n ""group"": null,\n ""lo...",2024-08-14 10:49:40.278,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,Z102N9L26D191S0,z102n9l26d191s0 - m214 - SC1 retail,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,NNNN,NaN,NaN,NaT,NaN
2,2024-09-03 17:05:31.186,n7Tktl6xJA2qDV8RcMpf,hsBtiG1y58ChVue6Mstj,{},EEL0bHavix9GmwdKvd0p,"{\n ""device"": ""92"",\n ""group"": null,\n ""loo...",2024-09-03 17:05:16.008,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,...,None,device missing rl06 smoke zone 1 rl06 sz1 77c ...,Z191N5L13D92S0,z191n5l13d92s0 - m214 - SC1 retail,device missing rl06 smoke zone 1 rl06 sz1 77c ...,??F?,NaN,NaN,NaT,NaN
3,2025-01-09 03:27:26.923,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},4RVo9ZSGMWFzHhvtUDcd,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2025-01-09 03:27:24.867,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,normal rl03 back of house rl03 z1 corridor mcp...,Z109N9L26D172S0,z109n9l26d172s0 - SC1 retail - m214,normal rl03 back of house rl03 z1 corridor mcp...,NNNN,NaN,NaN,NaT,NaN
4,2024-08-28 12:18:26.508,n7Tktl6xJA2qDV8RcMpf,5r2EosP2Vnb9cMJsQUA9,{},91S1PYh6HlGxdBqqMf9S,"{\n ""device"": ""136"",\n ""group"": null,\n ""lo...",2024-08-28 12:18:25.776,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,...,None,device missing node 1 ffcp modules eaf-b01-1 s...,Z801N1L3D136S0,z801n1l3d136s0 - m214 - SC1 retail,device missing node 1 ffcp modules eaf-b01-1 s...,??F?,NaN,NaN,NaT,NaN


In [47]:
df.head()

,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,IDENTIFIER,RECORDEDAT,SITEID,STATE,SYSTEMGENERATED,...,sensor_from_state,descriptor,detector,notification,raw,state,duration_alarm,duration_fault,duration_isolate,duration_preAlarm
0,2024-08-07 17:56:29.842,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},O0uFfYqPcl4PMU3uXxpe,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2024-08-07 17:56:28.957,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,initialising rl03 back of house rl03 z1 corrid...,Z109N9L26D172S0,z109n9l26d172s0 - m214 - SC1 retail,initialising rl03 back of house rl03 z1 corrid...,NNNN,NaN,NaN,NaT,NaN
1,2024-08-14 10:49:42.359,n7Tktl6xJA2qDV8RcMpf,yPHI0SRu7cRZ2YNmfDPG,{},xByYZSjb4uJqpLZkLwCa,"{\n ""device"": ""191"",\n ""group"": null,\n ""lo...",2024-08-14 10:49:40.278,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,Z102N9L26D191S0,z102n9l26d191s0 - m214 - SC1 retail,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,NNNN,NaN,NaN,NaT,NaN
2,2024-09-03 17:05:31.186,n7Tktl6xJA2qDV8RcMpf,hsBtiG1y58ChVue6Mstj,{},EEL0bHavix9GmwdKvd0p,"{\n ""device"": ""92"",\n ""group"": null,\n ""loo...",2024-09-03 17:05:16.008,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,...,None,device missing rl06 smoke zone 1 rl06 sz1 77c ...,Z191N5L13D92S0,z191n5l13d92s0 - m214 - SC1 retail,device missing rl06 smoke zone 1 rl06 sz1 77c ...,??F?,NaN,NaN,NaT,NaN
3,2025-01-09 03:27:26.923,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},4RVo9ZSGMWFzHhvtUDcd,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2025-01-09 03:27:24.867,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,normal rl03 back of house rl03 z1 corridor mcp...,Z109N9L26D172S0,z109n9l26d172s0 - SC1 retail - m214,normal rl03 back of house rl03 z1 corridor mcp...,NNNN,NaN,NaN,NaT,NaN
4,2024-08-28 12:18:26.508,n7Tktl6xJA2qDV8RcMpf,5r2EosP2Vnb9cMJsQUA9,{},91S1PYh6HlGxdBqqMf9S,"{\n ""device"": ""136"",\n ""group"": null,\n ""lo...",2024-08-28 12:18:25.776,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,...,None,device missing node 1 ffcp modules eaf-b01-1 s...,Z801N1L3D136S0,z801n1l3d136s0 - m214 - SC1 retail,device missing node 1 ffcp modules eaf-b01-1 s...,??F?,NaN,NaN,NaT,NaN


In [48]:
del df['STATE']
del df['IDENTIFIER']
del df['MESSAGE']


In [49]:
df.head()

,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,RECORDEDAT,SITEID,SYSTEMGENERATED,WILLSHOWINAPP,device,...,sensor_from_state,descriptor,detector,notification,raw,state,duration_alarm,duration_fault,duration_isolate,duration_preAlarm
0,2024-08-07 17:56:29.842,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},O0uFfYqPcl4PMU3uXxpe,2024-08-07 17:56:28.957,DO05xnc9uzEwZ1OvRGSP,False,True,172,...,None,initialising rl03 back of house rl03 z1 corrid...,Z109N9L26D172S0,z109n9l26d172s0 - m214 - SC1 retail,initialising rl03 back of house rl03 z1 corrid...,NNNN,NaN,NaN,NaT,NaN
1,2024-08-14 10:49:42.359,n7Tktl6xJA2qDV8RcMpf,yPHI0SRu7cRZ2YNmfDPG,{},xByYZSjb4uJqpLZkLwCa,2024-08-14 10:49:40.278,DO05xnc9uzEwZ1OvRGSP,False,True,191,...,None,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,Z102N9L26D191S0,z102n9l26d191s0 - m214 - SC1 retail,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,NNNN,NaN,NaN,NaT,NaN
2,2024-09-03 17:05:31.186,n7Tktl6xJA2qDV8RcMpf,hsBtiG1y58ChVue6Mstj,{},EEL0bHavix9GmwdKvd0p,2024-09-03 17:05:16.008,DO05xnc9uzEwZ1OvRGSP,False,True,92,...,None,device missing rl06 smoke zone 1 rl06 sz1 77c ...,Z191N5L13D92S0,z191n5l13d92s0 - m214 - SC1 retail,device missing rl06 smoke zone 1 rl06 sz1 77c ...,??F?,NaN,NaN,NaT,NaN
3,2025-01-09 03:27:26.923,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},4RVo9ZSGMWFzHhvtUDcd,2025-01-09 03:27:24.867,DO05xnc9uzEwZ1OvRGSP,False,True,172,...,None,normal rl03 back of house rl03 z1 corridor mcp...,Z109N9L26D172S0,z109n9l26d172s0 - SC1 retail - m214,normal rl03 back of house rl03 z1 corridor mcp...,NNNN,NaN,NaN,NaT,NaN
4,2024-08-28 12:18:26.508,n7Tktl6xJA2qDV8RcMpf,5r2EosP2Vnb9cMJsQUA9,{},91S1PYh6HlGxdBqqMf9S,2024-08-28 12:18:25.776,DO05xnc9uzEwZ1OvRGSP,False,True,136,...,None,device missing node 1 ffcp modules eaf-b01-1 s...,Z801N1L3D136S0,z801n1l3d136s0 - m214 - SC1 retail,device missing node 1 ffcp modules eaf-b01-1 s...,??F?,NaN,NaN,NaT,NaN


In [50]:
df['faultCode']


0                                                      None
1                                                      None
2            {'panel': 'AXIS', 'text': None, 'value': None}
3                                                      None
4            {'panel': 'AXIS', 'text': None, 'value': None}
                                ...                        
940078    {'panel': 'SIMPLEX', 'text': None, 'value': None}
940079    {'panel': 'SIMPLEX', 'text': None, 'value': None}
940080    {'panel': 'SIMPLEX', 'text': None, 'value': None}
940081                                                 None
940082                                                 None
Name: faultCode, Length: 940083, dtype: object

In [63]:
# Step 2: Convert JSON string to Python dict
df_json = df['DURATION'].fillna('{}').apply(json.loads)

# Step 3: Extract 'isolate' using safe .get()
df['duration_isolate'] = df_json.apply(lambda x: x.get('isolate'))

In [64]:
# Step 4: Safe parse
def safe_parse_duration(x):
    try:
        if pd.isnull(x):
            return pd.NaT
        return parse_duration(x)
    except:
        return pd.NaT

# Step 5: Apply parsing
df['duration_isolate'] = df['duration_isolate'].apply(safe_parse_duration)

In [65]:
print(df['duration_isolate'].notna().sum(), "parsed durations")
print(df['duration_isolate'].unique())



116795 parsed durations
[          'NaT' 174968573000000 604938119000000 ... 120254947000000
  22186931000000   9624483000000]


In [67]:
print(df['duration_isolate'].head(20))


0                       NaT
1                       NaT
2                       NaT
3                       NaT
4                       NaT
5                       NaT
6                       NaT
7                       NaT
8                       NaT
9                       NaT
10                      NaT
11                      NaT
12                      NaT
13                      NaT
14                      NaT
15   2 days 00:36:08.573000
16                      NaT
17                      NaT
18                      NaT
19                      NaT
Name: duration_isolate, dtype: timedelta64[ns]


In [70]:
df.head(20)

,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,RECORDEDAT,SITEID,SYSTEMGENERATED,WILLSHOWINAPP,device,...,sensor_from_state,descriptor,detector,notification,raw,state,duration_alarm,duration_fault,duration_isolate,duration_preAlarm
0,2024-08-07 17:56:29.842,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},O0uFfYqPcl4PMU3uXxpe,2024-08-07 17:56:28.957,DO05xnc9uzEwZ1OvRGSP,False,True,172,...,None,initialising rl03 back of house rl03 z1 corrid...,Z109N9L26D172S0,z109n9l26d172s0 - m214 - SC1 retail,initialising rl03 back of house rl03 z1 corrid...,NNNN,NaN,NaN,NaT,NaN
1,2024-08-14 10:49:42.359,n7Tktl6xJA2qDV8RcMpf,yPHI0SRu7cRZ2YNmfDPG,{},xByYZSjb4uJqpLZkLwCa,2024-08-14 10:49:40.278,DO05xnc9uzEwZ1OvRGSP,False,True,191,...,None,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,Z102N9L26D191S0,z102n9l26d191s0 - m214 - SC1 retail,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,NNNN,NaN,NaN,NaT,NaN
2,2024-09-03 17:05:31.186,n7Tktl6xJA2qDV8RcMpf,hsBtiG1y58ChVue6Mstj,{},EEL0bHavix9GmwdKvd0p,2024-09-03 17:05:16.008,DO05xnc9uzEwZ1OvRGSP,False,True,92,...,None,device missing rl06 smoke zone 1 rl06 sz1 77c ...,Z191N5L13D92S0,z191n5l13d92s0 - m214 - SC1 retail,device missing rl06 smoke zone 1 rl06 sz1 77c ...,??F?,NaN,NaN,NaT,NaN
3,2025-01-09 03:27:26.923,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},4RVo9ZSGMWFzHhvtUDcd,2025-01-09 03:27:24.867,DO05xnc9uzEwZ1OvRGSP,False,True,172,...,None,normal rl03 back of house rl03 z1 corridor mcp...,Z109N9L26D172S0,z109n9l26d172s0 - SC1 retail - m214,normal rl03 back of house rl03 z1 corridor mcp...,NNNN,NaN,NaN,NaT,NaN
4,2024-08-28 12:18:26.508,n7Tktl6xJA2qDV8RcMpf,5r2EosP2Vnb9cMJsQUA9,{},91S1PYh6HlGxdBqqMf9S,2024-08-28 12:18:25.776,DO05xnc9uzEwZ1OvRGSP,False,True,136,...,None,device missing node 1 ffcp modules eaf-b01-1 s...,Z801N1L3D136S0,z801n1l3d136s0 - m214 - SC1 retail,device missing node 1 ffcp modules eaf-b01-1 s...,??F?,NaN,NaN,NaT,NaN
5,2024-07-23 20:41:19.217,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},i3nf8E4FxlMW1ZzOy1h0,2024-07-23 20:41:17.453,DO05xnc9uzEwZ1OvRGSP,False,True,172,...,None,initialising rl03 back of house rl03 z1 corrid...,Z109N9L26D172S0,z109n9l26d172s0 - m214 - SC1 retail,initialising rl03 back of house rl03 z1 corrid...,NNNN,NaN,NaN,NaT,NaN
6,2024-12-11 02:38:35.703,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},TnXI50CNHrOJnPRkUWlu,2024-12-11 02:38:34.384,DO05xnc9uzEwZ1OvRGSP,False,True,172,...,None,initialising rl03 back of house rl03 z1 corrid...,Z109N9L26D172S0,z109n9l26d172s0 - SC1 retail - m214,initialising rl03 back of house rl03 z1 corrid...,NNNN,NaN,NaN,NaT,NaN
7,2024-11-06 16:15:09.196,n7Tktl6xJA2qDV8RcMpf,C29g2RaN96YNvHFaLpiE,{},tWM6FMeNqljcdcavheuO,2024-11-06 16:15:08.310,DO05xnc9uzEwZ1OvRGSP,False,True,None,...,None,mute,Z923N23,z923n23 - SC1 retail - m214,mute,???I,NaN,NaN,NaT,NaN
8,2024-09-10 22:24:11.867,n7Tktl6xJA2qDV8RcMpf,8hQnV3aU7CaTupG0w389,{},gcEyBbBoj5dh8kirqCSy,2024-09-10 22:24:11.867,DO05xnc9uzEwZ1OvRGSP,True,True,195,...,"{'maximum': None, 'minimum': None, 'normal': N...",active event expiry,Z44N1L2D195S0,detector id: z44n1l2d195s0 - m214 - SC1 retail,???nz44n1l2d195s0 active event expiry,???N,NaN,NaN,NaT,NaN
9,2024-08-11 17:10:44.711,n7Tktl6xJA2qDV8RcMpf,nSHXqKSwHbGZjNi6GYEJ,{},IAB0C9HOKcT7zcBOOw9o,2024-08-11 17:10:34.232,DO05xnc9uzEwZ1OvRGSP,False,True,30,...,None,normal rl04 smoke zone 3,Z133N9L23D30S0,z133n9l23d30s0 - m214 - SC1 retail,normal rl04 smoke zone 3,NNNN,NaN,NaN,NaT,NaN


In [72]:
# Timestamp conversion
df['ADDEDAT'] = pd.to_datetime(df['ADDEDAT'], errors='coerce')


In [73]:
df['RECORDEDAT'] = pd.to_datetime(df['RECORDEDAT'], errors='coerce')

In [74]:
df.head(10)

,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,RECORDEDAT,SITEID,SYSTEMGENERATED,WILLSHOWINAPP,device,...,sensor_from_state,descriptor,detector,notification,raw,state,duration_alarm,duration_fault,duration_isolate,duration_preAlarm
0,2024-08-07 17:56:29.842,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},O0uFfYqPcl4PMU3uXxpe,2024-08-07 17:56:28.957,DO05xnc9uzEwZ1OvRGSP,False,True,172,...,None,initialising rl03 back of house rl03 z1 corrid...,Z109N9L26D172S0,z109n9l26d172s0 - m214 - SC1 retail,initialising rl03 back of house rl03 z1 corrid...,NNNN,NaN,NaN,NaT,NaN
1,2024-08-14 10:49:42.359,n7Tktl6xJA2qDV8RcMpf,yPHI0SRu7cRZ2YNmfDPG,{},xByYZSjb4uJqpLZkLwCa,2024-08-14 10:49:40.278,DO05xnc9uzEwZ1OvRGSP,False,True,191,...,None,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,Z102N9L26D191S0,z102n9l26d191s0 - m214 - SC1 retail,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,NNNN,NaN,NaN,NaT,NaN
2,2024-09-03 17:05:31.186,n7Tktl6xJA2qDV8RcMpf,hsBtiG1y58ChVue6Mstj,{},EEL0bHavix9GmwdKvd0p,2024-09-03 17:05:16.008,DO05xnc9uzEwZ1OvRGSP,False,True,92,...,None,device missing rl06 smoke zone 1 rl06 sz1 77c ...,Z191N5L13D92S0,z191n5l13d92s0 - m214 - SC1 retail,device missing rl06 smoke zone 1 rl06 sz1 77c ...,??F?,NaN,NaN,NaT,NaN
3,2025-01-09 03:27:26.923,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},4RVo9ZSGMWFzHhvtUDcd,2025-01-09 03:27:24.867,DO05xnc9uzEwZ1OvRGSP,False,True,172,...,None,normal rl03 back of house rl03 z1 corridor mcp...,Z109N9L26D172S0,z109n9l26d172s0 - SC1 retail - m214,normal rl03 back of house rl03 z1 corridor mcp...,NNNN,NaN,NaN,NaT,NaN
4,2024-08-28 12:18:26.508,n7Tktl6xJA2qDV8RcMpf,5r2EosP2Vnb9cMJsQUA9,{},91S1PYh6HlGxdBqqMf9S,2024-08-28 12:18:25.776,DO05xnc9uzEwZ1OvRGSP,False,True,136,...,None,device missing node 1 ffcp modules eaf-b01-1 s...,Z801N1L3D136S0,z801n1l3d136s0 - m214 - SC1 retail,device missing node 1 ffcp modules eaf-b01-1 s...,??F?,NaN,NaN,NaT,NaN
5,2024-07-23 20:41:19.217,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},i3nf8E4FxlMW1ZzOy1h0,2024-07-23 20:41:17.453,DO05xnc9uzEwZ1OvRGSP,False,True,172,...,None,initialising rl03 back of house rl03 z1 corrid...,Z109N9L26D172S0,z109n9l26d172s0 - m214 - SC1 retail,initialising rl03 back of house rl03 z1 corrid...,NNNN,NaN,NaN,NaT,NaN
6,2024-12-11 02:38:35.703,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},TnXI50CNHrOJnPRkUWlu,2024-12-11 02:38:34.384,DO05xnc9uzEwZ1OvRGSP,False,True,172,...,None,initialising rl03 back of house rl03 z1 corrid...,Z109N9L26D172S0,z109n9l26d172s0 - SC1 retail - m214,initialising rl03 back of house rl03 z1 corrid...,NNNN,NaN,NaN,NaT,NaN
7,2024-11-06 16:15:09.196,n7Tktl6xJA2qDV8RcMpf,C29g2RaN96YNvHFaLpiE,{},tWM6FMeNqljcdcavheuO,2024-11-06 16:15:08.310,DO05xnc9uzEwZ1OvRGSP,False,True,None,...,None,mute,Z923N23,z923n23 - SC1 retail - m214,mute,???I,NaN,NaN,NaT,NaN
8,2024-09-10 22:24:11.867,n7Tktl6xJA2qDV8RcMpf,8hQnV3aU7CaTupG0w389,{},gcEyBbBoj5dh8kirqCSy,2024-09-10 22:24:11.867,DO05xnc9uzEwZ1OvRGSP,True,True,195,...,"{'maximum': None, 'minimum': None, 'normal': N...",active event expiry,Z44N1L2D195S0,detector id: z44n1l2d195s0 - m214 - SC1 retail,???nz44n1l2d195s0 active event expiry,???N,NaN,NaN,NaT,NaN
9,2024-08-11 17:10:44.711,n7Tktl6xJA2qDV8RcMpf,nSHXqKSwHbGZjNi6GYEJ,{},IAB0C9HOKcT7zcBOOw9o,2024-08-11 17:10:34.232,DO05xnc9uzEwZ1OvRGSP,False,True,30,...,None,normal rl04 smoke zone 3,Z133N9L23D30S0,z133n9l23d30s0 - m214 - SC1 retail,normal rl04 smoke zone 3,NNNN,NaN,NaN,NaT,NaN


In [76]:
# Check for missing timestamps (NaT)
print(df['ADDEDAT'].isna().sum(), "missing timestamps")

# Check the first few rows to verify timestamp format
print(df['ADDEDAT'].head())

# Check for invalid timestamps (NaT or outliers)
print(df['ADDEDAT'].describe())


0 missing timestamps
0   2024-08-07 17:56:29.842
1   2024-08-14 10:49:42.359
2   2024-09-03 17:05:31.186
3   2025-01-09 03:27:26.923
4   2024-08-28 12:18:26.508
Name: ADDEDAT, dtype: datetime64[ns]


C:\Users\User\AppData\Local\Temp\ipykernel_9520\685933736.py:8: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  print(df['ADDEDAT'].describe())


count                         940083
unique                        938202
top       2024-07-31 06:44:16.954000
freq                             421
first     2024-07-10 01:07:06.668000
last      2025-03-10 19:41:54.640000
Name: ADDEDAT, dtype: object


In [77]:
import os
print(os.getcwd())


C:\Users\User


In [87]:
# ISO conversion of Duration_fault
#: Extract 'fault' using safe .get()
df['duration_fault'] = df_json.apply(lambda x: x.get('fault'))

In [88]:
# Step 4: Safe parse
def safe_parse_duration(x):
    try:
        if pd.isnull(x):
            return pd.NaT
        return parse_duration(x)
    except:
        return pd.NaT

# Step 5: Apply parsing
df['duration_fault'] = df['duration_fault'].apply(safe_parse_duration)

In [89]:
print(df['duration_fault'].notna().sum(), "parsed durations")
print(df['duration_fault'].unique())

174226 parsed durations
[          'NaT' 604860610000000   1726313000000 ... 120252943000000
  22184715000000   9627810000000]


In [92]:
df.loc[30,'duration_fault']

Timedelta('7 days 00:01:00.610000')

In [94]:
# Save DataFrame as a CSV file
df.to_csv('Fire_Events_Cleaned.csv', index=False)
